In [38]:
import org.apache.spark.sql.SparkSession 
val spark = SparkSession
 .builder
 .appName("SparkSQLExampleApp")
.master("yarn")
.config("hive.merge.mapfiles", "false")
.config("hive.merge.tezfiles", "false")
.config("parquet.enable.summary-metadata", "false")
.config("spark.sql.parquet.mergeSchema", "false")
.config("hive.merge.smallfiles.avgsize", "160000000")
 .enableHiveSupport()
.config("hive.exec.dynamic.partition", "true")
.config("hive.exec.dynamic.partition.mode", "nonstrict")
.config("spark.sql.orc.impl", "native")
.config("spark.sql.orc.parquet.binaryAsString", "true")
.config("spark.sql.orc.parquet.writeLegacyFormat", "true")
 .getOrCreate()

// Path to data set 
val csvFile="LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/departuredelays.csv"


val df = spark.read.format("csv")
 .option("inferSchema", "true")
 .option("header", "true")
 .load(csvFile)


df.createOrReplaceTempView("us_delay_flights_tbl")


import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@36492084
csvFile: String = LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/departuredelays.csv
df: org.apache.spark.sql.DataFrame = [date: int, delay: int ... 3 more fields]


In [2]:
spark.sql("""SELECT distance, origin, destination 
FROM us_delay_flights_tbl WHERE distance > 1000 
ORDER BY distance DESC""").show(10)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



In [3]:
spark.sql("""SELECT date, delay, origin, destination 
FROM us_delay_flights_tbl 
WHERE delay > 120 AND ORIGIN = 'SFO' AND DESTINATION = 'ORD' 
ORDER by delay DESC""").show(10)


+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|2190925| 1638|   SFO|        ORD|
|1031755|  396|   SFO|        ORD|
|1022330|  326|   SFO|        ORD|
|1051205|  320|   SFO|        ORD|
|1190925|  297|   SFO|        ORD|
|2171115|  296|   SFO|        ORD|
|1071040|  279|   SFO|        ORD|
|1051550|  274|   SFO|        ORD|
|3120730|  266|   SFO|        ORD|
|1261104|  258|   SFO|        ORD|
+-------+-----+------+-----------+
only showing top 10 rows



In [4]:
spark.sql("""SELECT delay, origin, destination,
 CASE
 WHEN delay > 360 THEN 'Very Long Delays'
 WHEN delay > 120 AND delay < 360 THEN 'Long Delays'
 WHEN delay > 60 AND delay < 120 THEN 'Short Delays'
 WHEN delay > 0 and delay < 60 THEN 'Tolerable Delays'
 WHEN delay = 0 THEN 'No Delays'
 ELSE 'Early'
 END AS Flight_Delays
 FROM us_delay_flights_tbl
 ORDER BY origin, delay DESC""").show(10)


+-----+------+-----------+-------------+
|delay|origin|destination|Flight_Delays|
+-----+------+-----------+-------------+
|  333|   ABE|        ATL|  Long Delays|
|  305|   ABE|        ATL|  Long Delays|
|  275|   ABE|        ATL|  Long Delays|
|  257|   ABE|        ATL|  Long Delays|
|  247|   ABE|        ATL|  Long Delays|
|  247|   ABE|        DTW|  Long Delays|
|  219|   ABE|        ORD|  Long Delays|
|  211|   ABE|        ATL|  Long Delays|
|  197|   ABE|        DTW|  Long Delays|
|  192|   ABE|        ORD|  Long Delays|
+-----+------+-----------+-------------+
only showing top 10 rows



In [39]:
df.printSchema

root
 |-- date: integer (nullable = true)
 |-- delay: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)



In [8]:
df.select("date", "delay", "origin", "destination")
    .where((col("delay") > 120) && (col("origin") === "SFO") && (col("destination") === "ORD"))
    .orderBy(desc("delay"))
    .show(10, true)

+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|2190925| 1638|   SFO|        ORD|
|1031755|  396|   SFO|        ORD|
|1022330|  326|   SFO|        ORD|
|1051205|  320|   SFO|        ORD|
|1190925|  297|   SFO|        ORD|
|2171115|  296|   SFO|        ORD|
|1071040|  279|   SFO|        ORD|
|1051550|  274|   SFO|        ORD|
|3120730|  266|   SFO|        ORD|
|1261104|  258|   SFO|        ORD|
+-------+-----+------+-----------+
only showing top 10 rows



In [40]:

spark.sql("CREATE DATABASE learn_spark_db3")
spark.sql("USE learn_spark_db3")


res23: org.apache.spark.sql.DataFrame = []


In [57]:
//spark.sql("CREATE TABLE managed_us_delay_flights_tbl (date STRING, delay INT, distance INT, origin STRING, destination STRING)")
spark.sql("CREATE TABLE managed_us_delay_flights_tbl2 (date STRING, delay INT, distance INT, origin STRING, destination STRING) USING csv")

res37: org.apache.spark.sql.DataFrame = []


In [20]:
%sql
CREATE OR REPLACE GLOBAL TEMP VIEW us_origin_airport_SFO_global_tmp_view AS
 SELECT date, delay, origin, destination from us_delay_flights_tbl WHERE
 origin = 'SFO';

%sql
CREATE OR REPLACE TEMP VIEW us_origin_airport_JFK_tmp_view AS
 SELECT date, delay, origin, destination from us_delay_flights_tbl WHERE
 origin = 'JFK';


<console>: 4: error: ';' expected but ',' found.

In [24]:

spark.sql("CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
    USING json
    OPTIONS (
      path 'LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/json/*')")

<console>: 2: error: unclosed string literal

In [25]:
spark.sql("""CREATE TABLE us_delay_flights_tbl(date STRING, delay INT, 
 distance INT, origin STRING, destination STRING) 
 USING csv OPTIONS (PATH 
 'LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/departuredelays.csv')""")


res11: org.apache.spark.sql.DataFrame = []


In [31]:
spark.catalog.listTables()

res17: org.apache.spark.sql.Dataset[org.apache.spark.sql.catalog.Table] = [name: string, database: string ... 3 more fields]


In [34]:
val csv_file = "LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/departuredelays.csv"

csv_file: String = LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/departuredelays.csv


In [35]:
val schema="date STRING, delay INT, distance INT, origin STRING, destination STRING"


schema: String = date STRING, delay INT, distance INT, origin STRING, destination STRING


In [36]:
val flights_df = spark.read.csv(csv_file, schema=schema)


<console>: 33: error: overloaded method value csv with alternatives:

In [43]:
df.write.saveAsTable("managed_us_delay_flights_tbl")

org.apache.spark.sql.AnalysisException:  Table `managed_us_delay_flights_tbl` already exists.

In [45]:
spark.sql("""CREATE OR REPLACE GLOBAL TEMP VIEW us_origin_airport_SFO_global_tmp_view AS
 SELECT date, delay, origin, destination from us_delay_flights_tbl WHERE
 origin = 'SFO';
""")

res28: org.apache.spark.sql.DataFrame = []


In [46]:
spark.sql("""CREATE OR REPLACE TEMP VIEW us_origin_airport_JFK_tmp_view AS
 SELECT date, delay, origin, destination from us_delay_flights_tbl WHERE
 origin = 'JFK'
""")

res29: org.apache.spark.sql.DataFrame = []


In [49]:
spark.sql("""SELECT * FROM global_temp.us_origin_airport_SFO_global_tmp_view""").show(5, true)

+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|1011250|   55|   SFO|        JFK|
|1012230|    0|   SFO|        JFK|
|1010705|   -7|   SFO|        JFK|
|1010620|   -3|   SFO|        MIA|
|1010915|   -3|   SFO|        LAX|
+-------+-----+------+-----------+
only showing top 5 rows



In [50]:
spark.sql("""SELECT * FROM us_origin_airport_JFK_tmp_view""").show(5, true)

+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|1010900|   14|   JFK|        LAX|
|1011200|   -3|   JFK|        LAX|
|1011900|    2|   JFK|        LAX|
|1011700|   11|   JFK|        LAS|
|1010800|   -1|   JFK|        SFO|
+-------+-----+------+-----------+
only showing top 5 rows



In [52]:
spark.sql("""DROP VIEW IF EXISTS us_origin_airport_SFO_global_tmp_view;
""")

res34: org.apache.spark.sql.DataFrame = []


In [53]:
spark.sql("""DROP VIEW IF EXISTS us_origin_airport_JFK_tmp_view;
""")

res35: org.apache.spark.sql.DataFrame = []


In [61]:
spark.catalog.listDatabases().show(5, true)
spark.catalog.listTables().show(5, true)
spark.catalog.listColumns("us_delay_flights_tbl").show(5, true)

+---------------+----------------+--------------------+
|           name|     description|         locationUri|
+---------------+----------------+--------------------+
|        default|default database|file:/C:/Users/di...|
| learn_spark_db|                |file:/C:/Users/di...|
|learn_spark_db2|                |file:/C:/Users/di...|
|learn_spark_db3|                |file:/C:/Users/di...|
+---------------+----------------+--------------------+

+--------------------+---------------+-----------+---------+-----------+
|                name|       database|description|tableType|isTemporary|
+--------------------+---------------+-----------+---------+-----------+
|managed_us_delay_...|learn_spark_db3|       null|  MANAGED|      false|
|managed_us_delay_...|learn_spark_db3|       null|  MANAGED|      false|
|us_delay_flights_tbl|           null|       null|TEMPORARY|       true|
+--------------------+---------------+-----------+---------+-----------+

+-----------+-----------+--------+-----

In [59]:
val usFlightsDF = spark.sql("SELECT * FROM us_delay_flights_tbl")
val usFlightsDF2 = spark.table("us_delay_flights_tbl")


usFlightsDF: org.apache.spark.sql.DataFrame = [date: int, delay: int ... 3 more fields]
usFlightsDF2: org.apache.spark.sql.DataFrame = [date: int, delay: int ... 3 more fields]


In [63]:


val file= "C:/Users/diego.rodriguez/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/parquet/2010-summary.parquet/"
val df = spark.read.format("parquet").load(file)


file: String = C:/Users/diego.rodriguez/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/parquet/2010-summary.parquet/
df: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [64]:
spark.sql("""CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
 USING parquet
 OPTIONS (
 path "C:/Users/diego.rodriguez/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/parquet/2010-summary.parquet/" )
""")

res41: org.apache.spark.sql.DataFrame = []


In [65]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show()


+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|    1|
|       United States|            Ireland|  264|
|       United States|              India|   69|
|               Egypt|      United States|   24|
|   Equatorial Guinea|      United States|    1|
|       United States|          Singapore|   25|
|       United States|            Grenada|   54|
|          Costa Rica|      United States|  477|
|             Senegal|      United States|   29|
|       United States|   Marshall Islands|   44|
|              Guyana|      United States|   17|
|       United States|       Sint Maarten|   53|
|               Malta|      United States|    1|
|             Bolivia|      United States|   46|
|            Anguilla|      United States|   21|
|Turks and Caicos ...|      United States|  136|
|       United States|        Afghanistan|    2|
|Saint Vincent and..

In [66]:
df.write.format("parquet")
 .mode("overwrite")
 .option("compression", "snappy")
 .save("/tmp/data/parquet/df_parquet")


In [67]:
val file = "C:/Users/diego.rodriguez/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/json/*"
val df = spark.read.format("json").load(file)


file: String = C:/Users/diego.rodriguez/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/json/*
df: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [68]:
val file = "./LearningSparkV2-master/chapter3/data/prueba/"
val df = spark.read.format("parquet").load(file)

df.show(5, true)

+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|        CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|    Delay|
+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+------------

file: String = ./LearningSparkV2-master/chapter3/data/prueba/
df: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]


In [69]:
val file = "./LearningSparkV2-master/chapter3/data/pruebajson/"
val df = spark.read.format("json").load(file)

df.show(5, true)

+-------+--------------------+--------------------+---------+----+----------+--------------------+----------+----------------+-------------+----+---------+-------------+----------------------+--------------+--------------------+--------------------+---------+----------------+--------+-------------+-----------+------------------+------+--------------------------+--------+----------+-------+
|ALSUnit|             Address|       AvailableDtTm|Battalion| Box|  CallDate|CallFinalDisposition|CallNumber|        CallType|CallTypeGroup|City|    Delay|FinalPriority|FirePreventionDistrict|IncidentNumber|            Location|        Neighborhood|NumAlarms|OriginalPriority|Priority|        RowID|StationArea|SupervisorDistrict|UnitID|UnitSequenceInCallDispatch|UnitType| WatchDate|Zipcode|
+-------+--------------------+--------------------+---------+----+----------+--------------------+----------+----------------+-------------+----+---------+-------------+----------------------+--------------+-------

file: String = ./LearningSparkV2-master/chapter3/data/pruebajson/
df: org.apache.spark.sql.DataFrame = [ALSUnit: boolean, Address: string ... 26 more fields]


In [70]:
val file = "./LearningSparkV2-master/chapter3/data/pruebacsv/"
val df = spark.read.format("csv").load(file)

df.show(5, true)

+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------+--------------------+------------------+--------------------+--------------------+-------------+---------+
|       _c0|   _c1|           _c2|             _c3|       _c4|       _c5|                 _c6|                 _c7|                 _c8| _c9|   _c10|     _c11|       _c12|_c13|            _c14|    _c15|         _c16|   _c17|         _c18|     _c19|    _c20|                _c21|                _c22|              _c23|                _c24|                _c25|         _c26|     _c27|
+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+------

file: String = ./LearningSparkV2-master/chapter3/data/pruebacsv/
df: org.apache.spark.sql.DataFrame = [_c0: string, _c1: string ... 26 more fields]
